# NLP Project
Ciao gentaglia, qui a inizio notebook ci metto qualche info sul progetto, così non smattiamo per cercare le informazioni ogni volta.

You can find the ppt with details and requirements of the project <a href="https://docs.google.com/presentation/d/1TTN1H3GdnaswGXW63SuSvD4CsI7HB9lkYuwXRMQp2ks/edit?usp=sharing"> here</a>

You can find the FAQ page <a href="https://virtuale.unibo.it/mod/page/view.php?id=1405067"> here</a>

You can find the official webpage of the challenge <a href="https://lcs2.in/SemEval2024-EDiReF/"> here</a>



In [ ]:
GianM0027/NLP_course_project